# 🏎️ CarRacing AI 학습 노트북

이 노트북에서 PPO 알고리즘으로 자율주행 AI를 학습시킵니다.

**예상 소요 시간**: 30분~1시간 (GPU 사용 시)

---

## 1. GPU 설정 확인

**중요**: 런타임 > 런타임 유형 변경 > 하드웨어 가속기를 **GPU**로 설정하세요!

In [ ]:
# GPU 확인
!nvidia-smi

## 2. 필요한 패키지 설치

CarRacing 환경과 강화학습 라이브러리를 설치합니다.

In [ ]:
!pip install gymnasium[box2d] stable-baselines3[extra] shimmy moviepy pygame -q

## 3. 라이브러리 임포트

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import DummyVecEnv

print("✅ 라이브러리 임포트 완료!")

## 4. 학습 설정

In [ ]:
# 학습 설정
TOTAL_TIMESTEPS = 100000  # 총 학습 스텝 (더 늘리면 성능 향상)
CHECKPOINT_FREQ = 20000   # 체크포인트 저장 빈도
MODEL_NAME = "car_racing_driver"

# PPO 하이퍼파라미터
LEARNING_RATE = 0.0003
N_STEPS = 2048
BATCH_SIZE = 64
N_EPOCHS = 10

# 디렉토리 설정
CHECKPOINT_DIR = "./checkpoints"
LOG_DIR = "./logs"
MODEL_DIR = "./models"

# 디렉토리 생성
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("✅ 설정 완료!")

## 5. CarRacing 환경 생성

In [ ]:
print("🏎️ CarRacing 환경 초기화 중...")

# CarRacing-v3 환경 생성
env = gym.make("CarRacing-v3", continuous=True)
env = DummyVecEnv([lambda: env])  # 벡터 환경으로 래핑

print("✅ 환경 초기화 완료!")

## 6. PPO 모델 생성

In [ ]:
print("🤖 PPO 모델 생성 중...")

model = PPO(
    "CnnPolicy",                    # CNN 정책 (이미지 입력용)
    env,
    verbose=1,                      # 학습 진행 상황 출력
    tensorboard_log=LOG_DIR,        # TensorBoard 로그 저장
    learning_rate=LEARNING_RATE,
    n_steps=N_STEPS,
    batch_size=BATCH_SIZE,
    n_epochs=N_EPOCHS
)

print("✅ 모델 생성 완료!")

## 7. 체크포인트 콜백 설정

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=CHECKPOINT_FREQ,
    save_path=CHECKPOINT_DIR,
    name_prefix=MODEL_NAME
)

print("✅ 체크포인트 콜백 설정 완료!")

## 8. 학습 시작

이제 학습을 시작합니다! 진행 상황을 실시간으로 볼 수 있습니다.

**참고**: 
- 초반에는 점수가 음수입니다 (차가 제대로 못 움직임)
- 학습이 진행되면서 점점 점수가 올라갑니다
- 점수가 0을 넘어 양수가 되면 운전을 시작한 것입니다!

In [ ]:
print("="*50)
print("🏁 훈련 시작!")
print("="*50)
print(f"총 학습 스텝: {TOTAL_TIMESTEPS:,}")
print(f"체크포인트 저장 빈도: 매 {CHECKPOINT_FREQ:,} 스텝")
print(f"\n목표: 점수가 0점을 넘어 양수가 되면 운전을 시작한 것입니다!")
print("="*50 + "\n")

try:
    model.learn(
        total_timesteps=TOTAL_TIMESTEPS,
        callback=checkpoint_callback
    )
    
    print("\n" + "="*50)
    print("🎉 훈련 완료!")
    print("="*50)
    
except KeyboardInterrupt:
    print("\n\n⚠️ 훈련이 사용자에 의해 중단되었습니다.")

## 9. 최종 모델 저장

In [ ]:
final_model_path = os.path.join(MODEL_DIR, f"final_{MODEL_NAME}")
model.save(final_model_path)
print(f"\n💾 최종 모델 저장 완료: {final_model_path}")

# 환경 종료
env.close()
print("\n✅ 모든 작업 완료!")

## 10. 모델 다운로드 (선택사항)

학습된 모델을 로컬로 다운로드하려면 아래 셀을 실행하세요.

In [ ]:
from google.colab import files

# 최종 모델 다운로드
files.download(f"{final_model_path}.zip")
print("📥 모델 다운로드 시작!")

## 11. TensorBoard로 학습 과정 확인 (선택사항)

학습 진행 상황을 그래프로 확인할 수 있습니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

---

## 다음 단계

학습이 완료되었습니다! 이제 `colab_test.ipynb`를 실행하여 학습된 AI의 주행을 테스트해보세요.

### 학습을 더 개선하려면:
- `TOTAL_TIMESTEPS`를 늘려보세요 (예: 300000)
- 하이퍼파라미터를 조정해보세요
- 여러 환경을 병렬로 실행해보세요 (SubprocVecEnv)

### 문제가 발생한다면:
- GPU가 활성화되어 있는지 확인하세요
- 런타임을 재시작하고 다시 실행해보세요
- 체크포인트에서 학습을 재개할 수 있습니다
